In [1]:
import numpy as np
import copy

In [162]:
X_train_file = "C:/IITD/sem5/col774-ml/datasets/kannada_digits/neural_network_kannada/X_train.npy"
y_train_file = "C:/IITD/sem5/col774-ml/datasets/kannada_digits/neural_network_kannada/y_train.npy"
X_test_file = "C:/IITD/sem5/col774-ml/datasets/kannada_digits/neural_network_kannada/X_test.npy"
y_test_file = "C:/IITD/sem5/col774-ml/datasets/kannada_digits/neural_network_kannada/y_test.npy"

X_train, y_train = np.load(X_train_file), np.load(y_train_file)
X_test, y_test = np.load(X_test_file), np.load(y_test_file)

In [148]:
X_train = X_train[:15000]
y_train = y_train[:15000]

In [4]:
def one_hot_encode(y):
    ohe = np.zeros((y.size, y.max()+1))
    ohe[np.arange(y.size), y] = 1
    return ohe

In [163]:
m = X_train.shape[0]
n = 28 * 28
hidden_layers = [15]
r = 10
layers = hidden_layers.copy()
layers.append(r)

X_train = X_train.reshape((m, n)) # reshape
X_train = X_train / 255 # scale to 0-1
# y_train = one_hot_encode(y_train)

In [32]:
def g(z):
    """
        sigmoid(z)
    """
    return 1/(1 + np.exp(-z))

In [33]:
def netj(theta_j, x_j):
    return np.dot(theta_j.T, x_j)

In [103]:
def forward_propagation(x, theta):
    o = [np.zeros(l) for l in layers]

    for l in range(len(layers)):
        for j in range(layers[l]):
            if l == 0:
                o[l][j] = g(netj(theta[l][j], x))
            else:
                o[l][j] = g(netj(theta[l][j], o[l-1])) # use all outputs of prev layer as network is fully connected
    
    return o

In [10]:
def back_propagation(y, o, theta):
    """
        Arguments:
            y: class labels
            o: outputs of each layer
            theta: parameters
        Returns:
            deltas: deltas[l][j] for each layer l and perceptron j
    """
    deltas = [np.zeros(l) for l in layers]

    # output layer
    output_layer = -1
    delta = (y - o[output_layer]) * o[output_layer] * (1 - o[output_layer])
    deltas[output_layer] = delta

    # hidden layers
    for l in reversed(range(len(hidden_layers))):
        for j in range(hidden_layers[l]):
            deltas[l][j] = sum(deltas[l+1][dwn_nbr] * theta[l+1][dwn_nbr, j] * o[l][j] * (1 - o[l][j]) for dwn_nbr in range(layers[l+1]))
        
    return deltas

In [11]:
def total_cost(theta, X, Y):
    m = X.shape[0]
    error = 0
    for i in range(m):
        x, y = X[i], Y[i]
        o = forward_propagation(x, theta)
        error += np.sum((y - o[-1]) ** 2)
    return error / (2 * m)

In [133]:
theta0 = [np.random.randn(layers[0], n) * np.sqrt(2/(n))] + [np.random.randn(layers[l], layers[l-1]) * np.sqrt(2/layers[l-1]) for l in range(1, len(layers))]
# theta0 = [np.random.randn(layers[0], n) * 0.001] + [np.random.randn(layers[l], layers[l-1]) * 0.001 for l in range(1, len(layers))]
# theta0 = [np.zeros((layers[0], n+1))] + [np.zeros((layers[l], layers[l-1]+1)) for l in range(1, len(layers))]
# theta0 = [np.random.rand(layers[0], n+1)] + [np.random.rand(layers[l], layers[l-1]+1) for l in range(1, len(layers))]


# print(theta0[0][0])
for ind in [5, 6, 456]:
    xt = X_train[ind]
    print(ind)
    # print(xt)
    print(netj(theta0[0][0], xt))
    out0 = forward_propagation(xt, theta0)
    print(out0)
    print(netj(theta0[1][0], out0[0]))
total_cost(theta0, X_train, y_train)

5
0.552697555151562
[array([0.63476122, 0.55350336, 0.69467035, 0.53026362, 0.5342025 ,
       0.43449275, 0.5095823 , 0.3942845 , 0.56823072, 0.51860935,
       0.39584177, 0.51310929, 0.7072353 , 0.42276449, 0.46413004]), array([0.5627912 , 0.678119  , 0.71507255, 0.4537618 , 0.64135179,
       0.51538003, 0.30011677, 0.6488654 , 0.31167408, 0.45500852])]
0.2524978000853511
6
0.03797362268914106
[array([0.50949227, 0.41121447, 0.44858935, 0.56697234, 0.58073581,
       0.50376129, 0.4745573 , 0.48521124, 0.62170819, 0.53858699,
       0.38319595, 0.58885346, 0.5632412 , 0.47311895, 0.65047057]), array([0.54220668, 0.67135202, 0.71278775, 0.486648  , 0.62616902,
       0.58630912, 0.30626527, 0.65315984, 0.34641246, 0.43501986])]
0.16922945601884032
456
0.37219187702359735
[array([0.59198851, 0.44221212, 0.52080524, 0.55532562, 0.58720703,
       0.44337361, 0.57781215, 0.51273734, 0.41421409, 0.42528272,
       0.51040702, 0.51141345, 0.43262057, 0.54335195, 0.50748039]), array([0.50

1.4391922195860023

In [136]:
def gradient_descent(X_train, y_train, M, learning_rate, epsilon, max_epochs):
    """
        mini-batch SGD
    """
    epoch = 0
    t = 0
    k_repeats = 0
    # theta = [np.random.rand(layers[0], n+1)] + [np.random.rand(layers[l], layers[l-1]+1) for l in range(1, len(layers))]
    theta = [np.random.randn(layers[0], n) * np.sqrt(2/(n))] + [np.random.randn(layers[l], layers[l-1]) * np.sqrt(2/layers[l-1]) for l in range(1, len(layers))]
    # prev_theta = copy.deepcopy(theta)
    # prev_cost = -1
    # prev_cost = total_cost(theta, X_train, y_train)

    # theta = [np.zeros((layers[0], n+1))] + [np.zeros((layers[l], layers[l-1]+1)) for l in range(1, len(layers))]
    # print("theta", theta)

    while True:
        epoch += 1
        if epoch > max_epochs:
            return theta

        print("epoch", epoch, total_cost(theta, X_train, y_train))

        # shuffle
        indices = np.arange(m)
        np.random.shuffle(indices)
        X_train_e = X_train[indices]
        y_train_e = y_train[indices]

        for b in range(int(m/M)):
            t += 1
            # print("b", b, epoch, t)
            sum_J_theta_derivatives = [np.zeros((layers[0], n))] + [np.zeros((layers[l], layers[l-1])) for l in range(1, len(layers))]

            for i in range(b * M, (b+1) * M):
                x, y = X_train_e[i], y_train_e[i]
                o = forward_propagation(x, theta)
                deltas = back_propagation(y, o, theta)

                # print("c", y, o[-1], y - o[-1])
                # cost += np.sum((y - o[-1]) ** 2)
                # print("x", x.shape, x)
                # print("o", o)
                # print("deltas", deltas)

                # calculate J(theta) derivatives
                for l in range(len(layers)):
                    if l == 0:
                        # x_j = np.hstack((np.ones(1), x))
                        x_j = x
                    else:
                        # x_j = np.hstack((np.ones(1), o[l-1]))
                        x_j = o[l-1]
                    for j in range(layers[l]):
                        J_theta_derivative = - deltas[l][j] * x_j
                        sum_J_theta_derivatives[l][j] += J_theta_derivative

            # prev_theta = copy.deepcopy(theta)
            # print("sumJ", sum_J_theta_derivatives)
            # print("err", total_cost(theta, X_train, y_train))

            # update theta
            for l in range(len(layers)):
                for j in range(layers[l]):
                    theta[l][j] = theta[l][j] - learning_rate * (sum_J_theta_derivatives[l][j] / M)

            # cost = total_cost(theta, X_train, y_train)
            # # print("cost", prev_cost, cost, abs(prev_cost - cost))
            # if abs(prev_cost - cost) <= epsilon:
            #     k_repeats += 1
            # else:
            #     k_repeats = 0

            # if k_repeats > 1:
            #     print("converged")
            #     return theta
            # prev_cost = cost

            # print("w", theta)

            # break when stopping criteria meets
            # converged = True
            # theta_diff_all = 0
            # for l in range(len(layers)):
            #     # print("theta", theta[l], prev_theta[l])
            #     # print("theta_diff", abs(theta[l] - prev_theta[l]))
            #     theta_diff_all += np.sum(abs(theta[l] - prev_theta[l]))
            #     if not (abs(theta[l] - prev_theta[l]) <= epsilon).all():
            #         converged = False
            #         break
            # # print("stop", coverged, theta_diff_all)
            # if converged:
            #     k_repeats += 1
            # else:
            #     k_repeats = 0

            # if k_repeats > 2:
            #     print("converged!!")
            #     return theta


In [137]:
theta_opt = gradient_descent(X_train, y_train, 100, 0.5, 1e-6, 5)

epoch 1 1.2954430856441295
epoch 2 0.381024452313904
epoch 3 0.28697542670549486
epoch 4 0.21277853216797626
epoch 5 0.16651281976017654


In [138]:
total_cost(theta_opt, X_train, y_train)

0.13789600276230932

In [146]:
m_test = X_test.shape[0]
X_test = X_test.reshape((m_test, n)) # reshape
X_test = X_test / 255 # scale to 0-1
# y_test = one_hot_encode(y_test)
y_test

array([0, 1, 2, ..., 7, 8, 9], dtype=uint8)

In [140]:
theta_opt

[array([[ 0.09464465, -0.02737903,  0.01860194, ...,  0.03251293,
          0.02473725,  0.02264009],
        [-0.01868205, -0.12385706,  0.03245024, ...,  0.0730092 ,
         -0.06262206,  0.0559998 ],
        [-0.02562323,  0.06347721,  0.00437212, ...,  0.04151092,
          0.06693867,  0.05158981],
        ...,
        [-0.03669108,  0.00560304,  0.00119363, ..., -0.06088707,
          0.0116376 ,  0.03520283],
        [ 0.05112716, -0.00102293,  0.04138122, ...,  0.01205604,
          0.02369106,  0.02701591],
        [ 0.01359898, -0.04127114, -0.04589829, ..., -0.00516434,
          0.02983482, -0.04506464]]),
 array([[-2.56477056e-01, -1.61752914e-01, -1.31269219e+00,
         -5.58403040e-01, -1.06648291e+00, -8.48061974e-01,
          9.79730537e-01, -2.39385691e-01,  5.09355037e-01,
          4.20678246e-01, -1.32694100e-01, -7.89876914e-01,
         -1.97356853e+00,  3.44453002e-01, -1.26664578e-01],
        [-1.14423167e+00, -1.25474696e+00, -5.28335652e-01,
          5.

In [159]:
tt = 432

o_pred = forward_propagation(X_test[tt], theta_opt)

# print(o_pred[-1], y_train[tt])
# o_pred[-1]
print(np.argmax(o_pred[-1]), y_test[tt])

dd = y_train[tt] - o_pred[-1]

# print(y_train[230], o_pred[-1])

print(dd, dd ** 2, np.sum(dd ** 2))

2 2
[-0.03333155 -0.04925013  0.24252155 -0.08327136 -0.0731166  -0.24075464
 -0.05264166 -0.00671038 -0.0227034  -0.10264247] [1.11099215e-03 2.42557493e-03 5.88167022e-02 6.93411868e-03
 5.34603706e-03 5.79627984e-02 2.77114478e-03 4.50292486e-05
 5.15444484e-04 1.05354764e-02] 0.14646331835006932


In [165]:
acc = 0
for i in range(m):
    o_pred = forward_propagation(X_train[i], theta_opt)
    acc += int(np.argmax(o_pred[-1]) == y_train[i])

print(acc/m)

0.8969
